In [8]:
import pandas as pd
import io
import requests
import os
from collections import Counter
import json


In [27]:
df = pd.read_excel("financial_statements/17Q2-15Q3/BalanceSheet_ANV_24.xlsx", engine="openpyxl")
df.head()


,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Data Title,BALANCE SHEEET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Date Of Extract,2023-09-16 22:01:06.580000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
# Reset and drop Indexes
df = df.iloc[6:]
df = df.iloc[:-3]   #Drop the "powered by fiintrade"
df = df.reset_index(drop=True)
# Tranpose and reset the index again
df = df.transpose().reset_index(drop=True)
# Set the first row as headers
new_headers = df.iloc[0]
df = df[1:]
df.columns = new_headers
df = df.rename(columns={"ITEMS": "Quarter"})

# df = df.astype(str)
df.columns = df.columns.astype(str)

In [29]:
df.head()

,Quarter,TOTAL ASSETS,CURRENT ASSETS,Cash and cash equivalents,Cash,Cash equivalents,Short-term investments,Short-term investments,Provision for diminution,Held-to-maturity securities,...,Undistributed earnings,Beginning accumulated undistributed earnings,Current period undistributed earnings,Minority interests,Budget sources and other funds,Bonus and welfare funds (Before 2010),Budget sources and other funds,Funds used for fixed asset acquisitions,Minority Interest,TOTAL RESOURCES
1,Q1 2015,3272450781786,1757470205859,5875712011,5875712011,0,0,0,0,0,...,122565196878,0,122565196878,0,0,0,0,0,435328776,3272450781786
2,Q2 2015,3388617930256,1832805204414,17028384797,8098584797,8929800000,0,0,0,0,...,20450301117,1201789509,19248511608,467418359,0,0,0,0,0,3388617930256
3,Q3 2015,3330740724470,1803539038114,8744636996,8744636996,0,0,0,0,0,...,30172728873,1201789509,28970939364,472375005,0,0,0,0,0,3330740724470
4,Q4 2015,3276958780565,1817878538503,12707313325,12707313325,0,0,0,0,0,...,24882152794,11201789509,13680363285,477904963,0,0,0,0,0,3276958780565
5,Q1 2016,3242901585720,1795865953809,23405694166,23405694166,0,0,0,0,0,...,27013396626,21519354339,5494042287,476582453,0,0,0,0,0,3242901585720


**CHECK COMMON FIELDS**


In [13]:
# Specify the directory where the Excel files are located
directory = 'financial_statements/17Q2-15Q3'

# Specify the prefix of the files
prefix = 'IncomeStatement'

# Get a list of all Excel files in the directory with the specified prefix
files = [f for f in os.listdir(directory) if f.startswith(prefix) and (f.endswith('.xlsx') or f.endswith('.xls'))]

# Initialize a Counter to store field counts
field_counts = Counter()

# Read each file, update field counts, and transform it
for i, file in enumerate(files):
    try:
        df = pd.read_excel(os.path.join(directory, file))
        
        # Transformation step: normalize all string columns to lower case
        # Reset and drop Indexes
        df = df.iloc[6:]
        df = df.iloc[:-3]   #Drop the "powered by fiintrade"
        df = df.reset_index(drop=True)
        # Tranpose and reset the index again
        df = df.transpose().reset_index(drop=True)
        # Set the first row as headers
        new_headers = df.iloc[0]
        df = df[1:]
        df.columns = new_headers
        df = df.rename(columns={"ITEMS": "Quarter"})

        # df = df.astype(str)
        df.columns = df.columns.astype(str)
        
        # Update field counts
        field_counts.update(df.columns.tolist())
    except Exception:
        pass  # If an error occurs, ignore it and continue with the next file

# Find the most common fields
most_common_fields = field_counts.most_common()

# Convert the results to a JSON string
json_str = json.dumps(most_common_fields, indent=2)

# Write the JSON string to a file
with open(f'{prefix}_most_common_fields.json', 'w') as f:
    f.write(json_str)